# Import layers

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import sys
from pathlib import Path
import stat

In [2]:

p = Path()
LOCAL_PATH   = str(p.cwd().parent.parent)
PATH_DATASET = str(p.cwd().joinpath("dataset"))
LOCAL_PATH

'/home/gheorghe/Desktop/s_AI/shared_deep_neural_net/tensorflow/pipeline'

In [3]:
PATH_DATASET

'/home/gheorghe/Desktop/s_AI/shared_deep_neural_net/tensorflow/pipeline/csv_dataset/test/dataset'

In [4]:

# adding local_folder to the system path
sys.path.append(LOCAL_PATH)

In [5]:
from csv_dataset.code.csv_dataset import *

2025-08-11 18:41:23.734137: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-11 18:41:23.741633: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754926883.750071   57364 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754926883.752548   57364 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754926883.759187   57364 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# Rename filenames

In [ ]:
filenames = list(Path(PATH_DATASET).joinpath("inputs").glob('*'))
d_names = {}
for idx, filename in enumerate(filenames, 0):
    d_names[Path(filename.name).stem] = idx
d_names

In [ ]:

pipeline = PipilineCsvDataset(PATH_DATASET, shuffle=False)
for idx in range(pipeline.size_filenames()):
    input_filename, target_filename = pipeline.get_valid_filename()
    input_filename  = Path(input_filename)
    target_filename = Path(target_filename)
    stem = input_filename.stem
    new_stem = r"test_{}".format(d_names[stem])
    print("stem {}, new_stem {}".format(stem, new_stem))
    print("{}:in:  {}".format(idx, input_filename))
    print("{}:tar: {}".format(idx, target_filename))
    if (input_filename.is_file() == True):
        input_filename.rename(input_filename.with_stem(new_stem))
    if (target_filename.is_file() == True):
        target_filename.rename(target_filename.with_stem(new_stem))
    
    print("{}:in:  {}".format(idx, input_filename))
    print("{}:tar: {}".format(idx, target_filename))
    idx += 1


# PipilineCsvDataset

## PipilineCsvDataset shuffle

In [6]:
pipeline = PipilineCsvDataset(PATH_DATASET, True)
print()
print(pipeline)

{'description_path': 'description', 'input_path': 'inputs', 'input_suffix': '.png', 'output_suffix': '.csv', 'target_path': 'targets', 'height': 96, 'width': 128, 'channels': 3}
+++++++++++Not open files []
+++++++++++Not valid shape files []
+++++++++++All target filenames are valid

users [b'user_test' b'user_test_1' b'user_test_0'],
size_input_filenames 3, 
size_filenames       6,
is_filenames   False, 
shuffle        True,
        


I0000 00:00:1754926895.582221   57364 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21608 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


In [7]:
inputs = []
targets = []
users = []
err_count = 0
err_users = []
err_filenames = []
err_params = []
for idx in range(100000):
    try:
        input_filename, target_filename = pipeline.get_valid_filename()
        inputs.append(Path(input_filename).name)
        target_filename = Path(target_filename)
        #tmp = '{}/{}'.format(target_filename.parts[-2], target_filename.name)
        users.append(target_filename.parts[-2])
        targets.append(target_filename.name)
    except:
        err_count += 1
        err_users.append(pipeline.get_users())
        err_filenames.append(pipeline.get_filenames())
        err_params.append(pipeline.get_params())

pd_dataframe = pd.DataFrame({"inputs": inputs, "targets": targets, "users":users})

#pd_dataframe.sort_values(by=['inputs'])
#pd_dataframe.sort_values(by=['inputs', 'users'])
pd_dataframe

,inputs,targets,users
0,test_10.png,test_10.csv,user_test
1,test_7.png,test_7.csv,user_test
2,test_7.png,test_7.csv,user_test_0
3,test_7.png,test_7.csv,user_test
4,test_10.png,test_10.csv,user_test
...,...,...,...
99995,test_7.png,test_7.csv,user_test
99996,test_7.png,test_7.csv,user_test_0
99997,test_7.png,test_7.csv,user_test_1
99998,test_5.png,test_5.csv,user_test_1


In [8]:
print("err_count {}".format(err_count))
for user, filename, param in zip(err_users, err_filenames, err_params):
    print("user {}, filename {}, param {}".format(user, filename, param))

err_count 0


In [9]:
i = 0
stop = 3
pipeline.reset()
for x_input, y_target in pipeline():
    in_f, out_f = pipeline.get_act_filenames()
    print("filenames \nin :{}\nout:{}".format(in_f, out_f))
    print(x_input.shape)
    print(y_target)
    if (i >= stop):
        break
    i += 1

filenames 
in :/home/gheorghe/Desktop/s_AI/shared_deep_neural_net/tensorflow/pipeline/csv_dataset/test/dataset/inputs/test_7.png
out:/home/gheorghe/Desktop/s_AI/shared_deep_neural_net/tensorflow/pipeline/csv_dataset/test/dataset/targets/user_test_1/test_7.csv
(96, 128, 3)
          names  description  rating  coord x0  coord x1  coord y0  coord y1
0    all_within          NaN       0         0        -1         0        -1
1   all_without          NaN       0         0       128         0        96
2       Test_16  user_test_1       0        61        68         8        10
3             1  user_test_1       0        61        68        12        15
4             2  user_test_1       0        60        64        22        24
5             3  user_test_1       0        59        60        32        33
6             4  user_test_1       0        58        56        42        42
7             5  user_test_1       0        57        52        52        51
8             6  user_test_1      

In [11]:
print(pipeline)

users [b'user_test_1' b'user_test' b'user_test_0'],
size_input_filenames 3, 
size_filenames       6,
is_filenames   False, 
shuffle        True,
        


## PipilineCsvDataset not shuffle

In [12]:
Path(PATH_DATASET).owner()

'gheorghe'

In [13]:
Path(PATH_DATASET).stat()

os.stat_result(st_mode=16895, st_ino=24930315, st_dev=66309, st_nlink=5, st_uid=1000, st_gid=1000, st_size=4096, st_atime=1754925710, st_mtime=1705670500, st_ctime=1754925709)

In [14]:
Path(PATH_DATASET).stat()

os.stat_result(st_mode=16895, st_ino=24930315, st_dev=66309, st_nlink=5, st_uid=1000, st_gid=1000, st_size=4096, st_atime=1754925710, st_mtime=1705670500, st_ctime=1754925709)

In [15]:
Path(PATH_DATASET).chmod(stat.S_IREAD|stat.S_IRUSR|stat.S_IRGRP|stat.S_IROTH)
Path(PATH_DATASET).chmod(0o777)

In [18]:

pipeline = PipilineCsvDataset(PATH_DATASET, False)
user_names = pipeline.get_user_names()
print("user_names {}".format(user_names))
#print(filenames)
inputs = []
targets = []
users = []
pipeline.reset()
for idx in range(pipeline.size_filenames()*2):
    input_filename, target_filename = pipeline.get_valid_filename()
    inputs.append(Path(input_filename).name)
    target_filename = Path(target_filename)
    #tmp = '{}/{}'.format(target_filename.parts[-2], target_filename.name)
    users.append(target_filename.parts[-2])
    targets.append(target_filename.name)

pd_dataframe = pd.DataFrame({"inputs": inputs, "targets": targets, "users":users})

#pd_dataframe.sort_values(by=['inputs'])
#pd_dataframe.sort_values(by=['inputs', 'users'])
pd_dataframe

{'description_path': 'description', 'input_path': 'inputs', 'input_suffix': '.png', 'output_suffix': '.csv', 'target_path': 'targets', 'height': 96, 'width': 128, 'channels': 3}
+++++++++++Not open files []
+++++++++++Not valid shape files []
+++++++++++All target filenames are valid
user_names [b'user_test' b'user_test_1' b'user_test_0']


,inputs,targets,users
0,test_5.png,test_5.csv,user_test_1
1,test_7.png,test_7.csv,user_test_0
2,test_7.png,test_7.csv,user_test
3,test_5.png,test_5.csv,user_test
4,test_10.png,test_10.csv,user_test
5,test_7.png,test_7.csv,user_test_1
6,test_5.png,test_5.csv,user_test_1
7,test_7.png,test_7.csv,user_test_0
8,test_7.png,test_7.csv,user_test
9,test_5.png,test_5.csv,user_test


In [19]:
i = 0
stop = 4
pipeline.reset()
for x_input, y_target in pipeline():
    in_f, out_f = pipeline.get_act_filenames()
    print("filenames \nin :{}\nout:{}".format(in_f, out_f))
    print(x_input.shape)
    print(y_target)
    if (i >= stop):
        break
    i += 1

filenames 
in :/home/gheorghe/Desktop/s_AI/shared_deep_neural_net/tensorflow/pipeline/csv_dataset/test/dataset/inputs/test_5.png
out:/home/gheorghe/Desktop/s_AI/shared_deep_neural_net/tensorflow/pipeline/csv_dataset/test/dataset/targets/user_test/test_5.csv
(96, 128, 3)
          names description  rating  coord x0  coord x1  coord y0  coord y1
0    all_within         NaN       0         0        -1         0        -1
1   all_without         NaN       0         0       128         0        96
2       Test_13   user_test       0        61        68         4         7
3             1   user_test       0        61        68        10        12
4             2   user_test       1        62        69        11        13
5             3   user_test       2        63        70        12        14
6             4   user_test       3        64        71        13        15
7             5   user_test       4        65        72        14        16
8             6   user_test       5        66

In [20]:
print(pipeline)

users [b'user_test' b'user_test_1' b'user_test_0'],
size_input_filenames 3, 
size_filenames       6,
is_filenames   False, 
shuffle        False,
        


# PipilineCsvDataset Error

In [21]:

ERR_PATH_DATASET = str(p.cwd().joinpath("dataset_missed_target_files"))

## Input filenames do not have conection with target file

In [22]:

pipeline = PipilineCsvDataset(ERR_PATH_DATASET, True)
user_names = pipeline.get_users()
print("user_names {}".format(user_names))
#print(filenames)
inputs = []
targets = []
users = []
for idx in range(5):
    input_filename, target_filename = pipeline.valid_filename()
    inputs.append(Path(input_filename).name)
    target_filename = Path(target_filename)
    #tmp = '{}/{}'.format(target_filename.parts[-2], target_filename.name)
    users.append(target_filename.parts[-2])
    targets.append(target_filename.name)

pd_dataframe = pd.DataFrame({"inputs": inputs, "targets": targets, "users":users})

#pd_dataframe.sort_values(by=['inputs'])
#pd_dataframe.sort_values(by=['inputs', 'users'])
pd_dataframe

{'description_path': 'description', 'input_path': 'inputs', 'input_suffix': '.png', 'output_suffix': '.csv', 'target_path': 'targets', 'height': 96, 'width': 128, 'channels': 3}
+++++++++++Not open files []
+++++++++++Not valid shape files []
+++++++++++All target filenames are valid


TypeError: No logic conection from input filenames with target filenames
users ['user_test', 'user_test_1', 'user_test_0'], Invalid filenames: ['test_11', 'test_3', 'test_6', 'test_13', 'test_4', 'test_7', 'test_14', 'test_16', 'test_15', 'test_1', 'test_12', 'test_2', 'test_8', 'test_5', 'test_9', 'test_0', 'test_10']

## Set filenames do not have conection with target file

# PipilineCsvDataset set filenames

In [26]:
filenames = list(Path(PATH_DATASET).joinpath("inputs").glob('*'))
d_names = {}
for idx, filename in enumerate(filenames, 0):
    d_names[filename.name] = idx
print("dict names", d_names)

lst_name = ["test_10.png", "test_5.png", "test_7.png"]
lst_name.sort()
print("list names ", lst_name)

lst_user_filenames = list(map(lambda name: "user_test/{}".format(name), lst_name))
print("list user_names ", lst_user_filenames)


dict names {'test_6.png': 0, 'test_4.png': 1, 'test_7.png': 2, 'test_14.png': 3, 'test_15.png': 4, 'test_5.png': 5, 'test_not_suffix.jpg': 6, 'test_10.png': 7}
list names  ['test_10.png', 'test_5.png', 'test_7.png']
list user_names  ['user_test/test_10.png', 'user_test/test_5.png', 'user_test/test_7.png']


In [30]:

pipeline = PipilineCsvDataset(PATH_DATASET, False, lst_user_filenames)
user_names = pipeline.get_user_names()
print("user_names {}".format(user_names))
print(pipeline)
inputs = []
targets = []
users = []
for idx in range(5):
    input_filename, target_filename = pipeline.get_valid_filename()
    inputs.append(Path(input_filename).name)
    target_filename = Path(target_filename)
    #tmp = '{}/{}'.format(target_filename.parts[-2], target_filename.name)
    users.append(target_filename.parts[-2])
    targets.append(target_filename.name)

pd_dataframe = pd.DataFrame({"inputs": inputs, "targets": targets, "users":users})

#pd_dataframe.sort_values(by=['inputs'])
#pd_dataframe.sort_values(by=['inputs', 'users'])
pd_dataframe

{'description_path': 'description', 'input_path': 'inputs', 'input_suffix': '.png', 'output_suffix': '.csv', 'target_path': 'targets', 'height': 96, 'width': 128, 'channels': 3}
+++++++++++Not open files []
+++++++++++Not valid shape files []
+++++++++++All target filenames are valid
user_names [b'user_test']
users [b'user_test'],
size_input_filenames 3, 
size_filenames       3,
is_filenames   True, 
shuffle        False,
        


,inputs,targets,users
0,test_10.png,test_10.csv,user_test
1,test_5.png,test_5.csv,user_test
2,test_7.png,test_7.csv,user_test
3,test_10.png,test_10.csv,user_test
4,test_5.png,test_5.csv,user_test


In [31]:
i = 0
stop = 1000
for x_input, y_target in pipeline():
    print(x_input.shape)
    #print(target)
    if (i >= stop):
        break
    i += 1

(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)
(96, 128, 3)